In [ ]:
scd = {}
scd['ACR_35'] = [12]
scd['ACR_29'] = [2, 16]
scd['ACR_30'] = [9]

In [ ]:
from acr.utils import SOM_BLUE, NNXR_GRAY

MAIN_EXP = 'swi'
SUBJECT_TYPE = 'som'
MAIN_COLOR = SOM_BLUE

In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import warnings
import pingouin as pg
from scipy.stats import shapiro, normaltest

warnings.filterwarnings('ignore')
probe_ord = ['NNXr', 'NNXo']
hue_ord = [NNXR_GRAY, MAIN_COLOR]

from acr.plots import pub, lrg

plt.style.use('fast')
pub()
plt.rcParams['xtick.bottom'] = False
#--------------------------------- Import Publication Functions ---------------------------------#
pub_utils = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
from pub_utils import *
data_agg = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
from data_agg import *

In [ ]:
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject, exp in zip(subjects, exps):
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp)
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=True, extra_rebounds=False)

In [ ]:
#load all raw bp dfs
bp_dfs = {}
for sub, exp in zip(subjects, exps):
    bp = load_raw_bp_df(sub, exp)
    #bp = acr.hypnogram_utils.label_df_with_hypno_conditions(bp, hyp_dicts[sub])
    #bp = data_agg._label_12_hr_baseline(bp)
    bp_dfs[sub] = bp

In [ ]:
reldfs = []
for subject in subjects:
    print(subject)
    subdf = bp_dfs[subject]
    subdf = acr.hypnogram_utils.label_df_with_hypno_conditions(subdf, hyp_dicts[subject])
    print('Done conditions')
    subdf = acr.hypnogram_utils.label_df_with_states(subdf, full_hyps[subject])
    subdf = acr.hypnogram_utils.label_df_with_full_bl(subdf)
    subdf = make_raw_bp_df_relative_to_baseline(subdf, col_to_use='full_bl', value_to_use='True', state_to_use='NREM', method='mean')
    print('Done states/bl')
    reldfs.append(subdf)
relbp = pl.concat(reldfs)

In [ ]:
# drop the known bad channels
relbp = drop_sub_channels(relbp, scd)

In [ ]:
plt.rcdefaults()
# quick data check to make sure all remaining channels are good
g = sns.catplot(data=relbp.filter(pl.col('band') == 'delta').filter(pl.col('condition')=='rebound').to_pandas(), 
                x='channel', 
                y='bandpower_rel', 
                hue='store', 
                kind='box',
                row='subject',
                row_order=subjects,
                showfliers=False,
                showmeans=True, meanprops={"marker": "_", 'markerfacecolor':'gold','markeredgecolor':'gold','markersize':'24', "markeredgewidth": "3"},
                height=10,
                aspect=3,
                hue_order=probe_ord, 
                palette=hue_ord
                )

for ax in g.axes.flat:
    ax.set_ylim(0, 5)

acr.plots.lrg()

In [ ]:
rebbp = relbp.filter(pl.col('condition') == 'rebound')
reb_avgs = rebbp.group_by(['subject', 'store', 'band']).mean().to_pandas()
reb_avgs = sub_regions_to_df(reb_avgs)

# SWA, simple average of all channels

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'ALL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=(3.5, 4))

ax.set_ylim(1, 2) # y limits
ax.set_xlim(0.30, 0.70) # x limits
ax.set_yticks([1, 1.5, 2]) # y ticks
#ax.set_yticks([1.25, 1.75], minor=True) # minor y ticks

plt.tight_layout(pad=0.2)
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = False

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')
stats = pg.ttest(nnxr, nnxo, paired=True)
hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')
if write:
    # ===== Write Stats =====
    stats_name = f'REB-SWA__{SUBJECT_TYPE.upper()}__{SUBJECT_GROUP}__{CHANNEL_GROUP}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = False
BAND = 'delta'
SUBJECT_GROUP = 'ALL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}__NORMALIZED'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

nnxr_norm = nnxr/nnxr
nnxo_norm = nnxo/nnxr

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr_norm, nnxo_norm, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True, fsize=(3.5, 4), lineplot_width=3.5)


ax.set_ylim(0.8, 1.01) # y limits
ax.set_xlim(0.35, 0.70) # x limits
ax.set_yticks([0.8, 0.9, 1]) # y ticks

plt.tight_layout(pad=0.2)
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'FRONTAL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND').query('region == "frontal"')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=False, fsize=(3.5, 4), lineplot_width=3.5)

ax.set_ylim(1, 2) # y limits
ax.set_xlim(0.30, 0.70) # x limits
ax.set_yticks([1, 1.5, 2]) # y ticks

plt.tight_layout(pad=0.2)
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
#agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')
stats = pg.ttest(nnxr, nnxo, paired=True)
hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')
if write:
    # ===== Write Stats =====
    stats_name = f'REB-SWA__{SUBJECT_TYPE.upper()}__{SUBJECT_GROUP}__{CHANNEL_GROUP}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
    
stats

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'FRONTAL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}__NORMALIZED'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND').query('region == "frontal"')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

nnxr_norm = nnxr/nnxr
nnxo_norm = nnxo/nnxr

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr_norm, nnxo_norm, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True, fsize=(3.5, 4), lineplot_width=3.5)


ax.set_ylim(0.9, 1.01) # y limits
ax.set_xlim(0.35, 0.70) # x limits
ax.set_yticks([0.9, 1]) # y ticks

plt.tight_layout(pad=0.2)
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'PARIETAL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND').query('region == "parietal"')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=False, fsize=(3.5, 4), lineplot_width=3.5)

ax.set_ylim(1, 1.4) # y limits
ax.set_xlim(0.30, 0.70) # x limits
ax.set_yticks([1, 1.2, 1.4]) # y ticks

plt.tight_layout(pad=0.2)
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
#agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')
stats = pg.ttest(nnxr, nnxo, paired=True)
hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')
if write:
    # ===== Write Stats =====
    stats_name = f'REB-SWA__{SUBJECT_TYPE.upper()}__{SUBJECT_GROUP}__{CHANNEL_GROUP}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'PARIETAL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}__NORMALIZED'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND').query('region == "parietal"')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

nnxr_norm = nnxr/nnxr
nnxo_norm = nnxo/nnxr

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr_norm, nnxo_norm, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True, fsize=(3.5, 4), lineplot_width=3.5)

ax.set_ylim(0.8, 1.01) # y limits
ax.set_xlim(0.35, 0.70) # x limits
ax.set_yticks([0.8, 0.9, 1]) # y ticks
ax.set_yticks([0.85, 0.95], minor=True) # minor y ticks

plt.tight_layout(pad=0.2)
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)


## quick check on parietal subset

In [ ]:
match_subs = [
    'ACR_25',
    'ACR_39', 
    'ACR_40', 
    'ACR_41', 
    'ACR_44', 
    ]

# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'MATCHED_PARIETAL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND').query('subject in @match_subs')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=False, fsize=(3.5, 4), lineplot_width=3.5)

ax.set_xlim(0.30, 0.70) # x limits
ax.set_yticks([1, 1.2, 1.4]) # y ticks
ax.set_ylim(1.08, 2.0) # y limits

plt.tight_layout(pad=0.2)
#plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = False

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
#agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
#print(f'd,agostino_p-value: {agostino_p}')
stats = pg.ttest(nnxr, nnxo, paired=True)
hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')
if write:
    stats_name = f'REB-SWA__{SUBJECT_TYPE.upper()}__{SUBJECT_GROUP}__{CHANNEL_GROUP}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
stats

# Full Spectrograms

In [ ]:
reb_spgs = {}
for subject, exp in zip(subjects, exps):
    if subject in scd:
        to_drop = scd[subject]
    else:
        to_drop=None
    reb_spgs[subject] = create_reb_spg(subject, exp, drop_chans=to_drop)
full_df = pd.concat(reb_spgs.values())
full_means = full_df.groupby(['store', 'frequency']).mean().reset_index()

In [ ]:
from statsmodels.stats.multitest import multipletests
# statistical-tests for all frequencies
t_tests = {}
freqs = []
p_vals = []
for freq in full_df['frequency'].unique():
    freqs.append(freq)
    freq_df = full_df.loc[full_df['frequency'] == freq].sort_values(['store', 'subject'])
    nnxr = freq_df.query('store == "NNXr"')['power_rel'].values
    nnxo = freq_df.query('store == "NNXo"')['power_rel'].values
    
    # will use wilcoxon across all frequencies to avoid differing normality assumptions across different frequencies, etc
    stat = pg.wilcoxon(nnxr, nnxo)
    p_vals.append(stat['p-val'][0])

In [ ]:
# Correct for multiple Comparisons using Benjamini–Hochberg False discovery rate correction 
rej, p_fdr, _, _ = multipletests(p_vals, alpha=0.05, method='fdr_bh')

for p_cor, freq in zip(p_fdr, freqs):
    if p_cor<0.05:
        print(round(freq, 1), round(p_cor, 3))
    #print(round(freq, 1), round(p_cor, 3))


In [ ]:
#significant at 0.5-3Hz after correction for multiple comparisons

In [ ]:
# =============================
# ===== Write Source Data =====
# =============================
dfs = []
for freq in full_df['frequency'].unique():
    freqs.append(freq)
    freq_df = full_df.loc[full_df['frequency'] == freq].sort_values(['store', 'subject'])
    nnxr = freq_df.query('store == "NNXr"')['power_rel'].values
    nnxo = freq_df.query('store == "NNXo"')['power_rel'].values
    f = round(freq, 1)
    fdf = pd.DataFrame({'freq_bin': f, 'contra_control_power': nnxr, 'optrode_power': nnxo, 'subject': np.arange(len(nnxr))})
    dfs.append(fdf)
spg_source_dat = pd.concat(dfs)
pub_utils.write_source_data(spg_source_dat, 'SOM_full_spg_source_data')

In [ ]:
SUBJECT_GROUP = 'ALL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = 'spg_rebound'

fig_path = f'{PAPER_FIGURE_ROOT}/spectral/{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{MAIN_EXP}--{FIG_ID}.png'
# --------------------------------------------------------------------------------------------------

plt.rcdefaults()  # Reset to matplotlib defaults
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = True

f, ax = plt.subplots(1, 1, figsize=(4.5, 3))
sns.lineplot(data=full_df, x='frequency', y='power_rel', hue='store', ax=ax, palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, errorbar='se', lw=4)

ax.set_xlim(0, 20)  
ax.set_ylim(0.8, 1.6)
ax.set_yticks([0.8, 1, 1.2, 1.4, 1.6])
ax.set_xticks([0, 5, 10, 15, 20])

#remove the legend
ax.legend_.remove()
#ax.axhline(y=1.59, xmin=0.09, xmax=0.15)
#ax.axvspan(0.3, 2.4, ymin=0.985, ymax=1.0)
#ax.axvspan(2.6, 3.5, ymin=0.985, ymax=1.0)

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
SUBJECT_GROUP = 'ALL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = 'spg_rebound__SIGNIFICANCE_MARKERS'

fig_path = f'{PAPER_FIGURE_ROOT}/spectral/{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{MAIN_EXP}--{FIG_ID}.svg'
# --------------------------------------------------------------------------------------------------

plt.rcdefaults()  # Reset to matplotlib defaults
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = True

f, ax = plt.subplots(1, 1, figsize=(4.5, 3))
sns.lineplot(data=full_df, x='frequency', y='power_rel', hue='store', ax=ax, palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, errorbar='se', lw=4)

ax.set_xlim(0, 20)  
ax.set_ylim(0.8, 1.6)
ax.set_yticks([0.8, 1, 1.2, 1.4, 1.6])
ax.set_xticks([0, 5, 10, 15, 20])

#remove the legend
ax.legend_.remove()

ax.hlines(1.5, 0.5, 3)

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# Recovery

In [ ]:
plt.rcdefaults()
acr.plots.supl()
nbroot = f'{acr.utils.PAPER_FIGURE_ROOT}/swa/recovery'

In [ ]:
hd_rebs = {}
for subject, exp in zip(subjects, exps):
    hd_rebs[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=True, extra_rebounds=True)

In [ ]:
recdfs = []
for subject in subjects:
    hds_to_use = hd_rebs[subject]
    cons = ['rebound', 'reb2', 'reb3', 'reb4', 'reb5']
    cons_in_dict = list(hds_to_use.keys())
    for c in cons_in_dict:
        if c not in cons:
            del hds_to_use[c]
    subdf = relbp.filter(pl.col('subject') == subject)
    reb_start = hyp_dicts[subject]['rebound']['start_time'].min()
    subdf = subdf.ts(reb_start, reb_start+pd.Timedelta('48h')).sort('datetime')
    subdf = acr.hypnogram_utils.label_df_with_hypno_conditions(subdf, hd_rebs[subject], label_col='reb_condition')
    #subdf = acr.hypnogram_utils.label_df_with_full_bl(subdf)
    recdfs.append(subdf)
recdf = pl.concat(recdfs)

In [ ]:
for subject in subjects:
    df = relbp.filter(pl.col('subject') == subject)
    reb_start = df.filter(pl.col('condition') == 'rebound')['datetime'].to_pandas().min()
    end = df['datetime'].to_pandas().max()
    f, ax = plt.subplots(1, 1, figsize=(28, 8))
    sns.lineplot(data=df.filter(pl.col('datetime') > reb_start).prb('NNXo').ch(8).filter(pl.col('band') == 'delta').to_pandas(), x='datetime', y='bandpower_rel', hue='store', ax=ax)
    diff = (end-reb_start).total_seconds()/3600
    ax.set_title(f'{subject} --- {diff} hours')
    plt.show()

In [ ]:
delta = relbp.filter(pl.col('band') == 'delta')
delta = delta.sort('datetime')
d2min = delta.group_by_dynamic(index_column='datetime', every='1m', closed='left', group_by=['subject', 'store'], start_by='datapoint').agg(pl.col('bandpower_rel').mean())

In [ ]:
dfs = []
for subject in subjects:
    df = d2min.filter(pl.col('subject') == subject)
    df = acr.hypnogram_utils.label_df_with_states(df, full_hyps[subject])
    dfs.append(df)
d2min = pl.concat(dfs)

In [ ]:
dfs = []
for subject in subjects:
    rs = hyp_dicts[subject]['rebound']['start_time'].min()
    df = d2min.filter(pl.col('subject') == subject).ts(rs, rs+pd.Timedelta(hours=6)).st('NREM')
    for probe in ['NNXo', 'NNXr']:
        pdf = df.prb(probe)
        bouts = np.arange(0, len(pdf), 1)
        pdf = pdf.with_columns(bout=pl.lit(bouts))
        dfs.append(pdf)
dreb = pl.concat(dfs)
dreb_bouts = dreb.filter(pl.col('bout')<=240).group_by(['subject', 'bout', 'store']).agg(pl.col('bandpower_rel').mean()).sort(['subject', 'store', 'bout'])

In [ ]:
nxo = dreb_bouts.sort(['subject', 'bout']).prb('NNXo')['bandpower_rel'].to_numpy()
nxr = dreb_bouts.sort(['subject', 'bout']).prb('NNXr')['bandpower_rel'].to_numpy()
bouts = dreb_bouts.sort(['subject', 'bout']).prb('NNXr')['bout'].to_numpy()
subs = dreb_bouts.sort(['subject', 'bout']).prb('NNXr')['subject'].to_numpy()
diffdf = pd.DataFrame({'subject': subs, 'nxo': nxo, 'nxr': nxr, 'bout':bouts})
diffdf['relbp'] = diffdf['nxo']/diffdf['nxr']

In [ ]:
fig_id = 'delta_rebound_recovery_over_time'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

f, ax = plt.subplots(1, 1, figsize=(16, 8))
sns.lineplot(data=dreb_bouts.to_pandas(), x='bout', y='bandpower_rel', hue='store', hue_order=['NNXr', 'NNXo'], palette=hue_ord, ax=ax, linewidth=4, errorbar='se')
ax.axhline(1.0, color='red', linestyle='--')
ax.set_xlim(0, 180)
ax.set_ylabel('')
ax.set_xlabel('Time (min)')
ax.legend().remove()
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
for cdd in ['rebound', 'reb2', 'reb3', 'reb4', 'reb5']:
    durs = [hd_rebs[subject][cdd].duration.sum().total_seconds() for subject in subjects]
    print(cdd, np.mean(durs))

In [ ]:
first = recdf.filter(pl.col('reb_condition') == 'rebound').filter(pl.col('band')=='delta')
fig_id = 'delta_rebound_recovery_hr1'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

fm = first.group_by(['subject', 'store']).agg(pl.col('bandpower_rel').mean()).sort(['subject', 'store'])
nnxo = fm.prb('NNXo')['bandpower_rel'].to_numpy()
nnxr = fm.prb('NNXr')['bandpower_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr/nnxr, nnxo/nnxr, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
ax.set_xticklabels(['Contra. Control', 'Optrode'])
ax.set_ylim(0.82, 1.05)
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
first = recdf.filter(pl.col('reb_condition') == 'reb3').filter(pl.col('band')=='delta')
fig_id = 'delta_rebound_recovery_hr3'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

fm = first.group_by(['subject', 'store']).agg(pl.col('bandpower_rel').mean()).sort(['subject', 'store'])
nnxo = fm.prb('NNXo')['bandpower_rel'].to_numpy()
nnxr = fm.prb('NNXr')['bandpower_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr/nnxr, nnxo/nnxr, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
ax.set_xticklabels(['Contra. Control', 'Optrode'])
ax.set_ylim(0.82, 1.05)
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
for subject in subjects:
    df = dreb_bouts.filter(pl.col('subject') == subject)
    f, ax = plt.subplots(1, 1, figsize=(28, 8))
    sns.lineplot(data=df.to_pandas(), x='bout', y='bandpower_rel', hue='store', hue_order=['NNXr', 'NNXo'], palette=hue_ord, ax=ax)
    ax.set_title(f'{subject}')
    plt.show()

# SWA during Induction

In [ ]:
nbroot = f'{acr.utils.PAPER_FIGURE_ROOT}/swa'

In [ ]:
plt.rcdefaults()
acr.plots.supl()

In [ ]:
# try to clearn out as much obvious artifact from the wake data as possible here
conds = ['early_sd','late_sd', 'stim']
sdf = relbp.filter(pl.col('condition').is_in(conds)).filter(pl.col('band')=='delta')
sdf = sdf.filter(~((pl.col('condition').is_in(['early_sd', 'late_sd']))&(pl.col('bandpower_rel') > 1.5)))
sdf = sdf.filter(~((pl.col('store') == 'NNXr')&(pl.col('bandpower_rel') > 1.5)))
sdf = sdf.filter(~((pl.col('store') == 'NNXo')&(pl.col('bandpower_rel') > 10)))
sdfm = sdf.group_by(['condition','subject', 'store', 'channel']).agg(pl.col('bandpower_rel').mean())

In [ ]:
for sub in subjects:
    subdf = sdfm.filter(pl.col('subject') == sub)
    f, ax = plt.subplots(2, 1, figsize=(18, 6))    
    sns.barplot(data=subdf.prb('NNXo').to_pandas(), x='channel', y='bandpower_rel', hue='condition', hue_order=conds, ax=ax[0], palette=['gray', NNXR_GRAY, MAIN_COLOR])
    sns.barplot(data=subdf.prb('NNXr').to_pandas(), x='channel', y='bandpower_rel', hue='condition', hue_order=conds, ax=ax[1], palette=['gray', NNXR_GRAY, 'black'])
    ax[0].set_title(f'{sub}-NNXo')
    ax[1].set_title(f'{sub}-NNXr')
    ax[0].axhline(y=1, color='red', linestyle='--')
    ax[1].axhline(y=1, color='red', linestyle='--')
    ax[0].set_ylim(0, 3)
    ax[1].set_ylim(0, 3)
    ax[0].legend().remove()
    ax[1].legend().remove()
    plt.show()

In [ ]:
stimdf = sdf.filter(pl.col('condition') == 'stim')
stimmeans = stimdf.group_by(['subject', 'store']).agg(pl.col('bandpower_rel').mean()).sort(['subject', 'store'])

In [ ]:
fig_id = 'delta_during_stim'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

nnxo = stimmeans.filter(pl.col('store') == 'NNXo')['bandpower_rel'].to_numpy()
nnxr = stimmeans.filter(pl.col('store') == 'NNXr')['bandpower_rel'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.show()
f, ax = acr.plots.gen_paired_boxplot(nnxr/nnxr, nnxo/nnxr, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
ax.set_xticklabels(['Contra. Control', 'Optrode'])
#ax.set_ylim(0.7, 4.1)
plt.show()

f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
conds = ['early_sd','late_sd', 'stim']
sdf = relbp.filter(pl.col('condition').is_in(conds)).filter(pl.col('band')=='delta')
sdf = sdf.filter(~((pl.col('condition').is_in(['early_sd', 'late_sd']))&(pl.col('bandpower_rel') > 1.5)))
sdf = sdf.filter(pl.col('channel') >= 9) #using only the deep channels here to avoid the inevitable omnetics-caused artifact on the superficial channels during periods of intense movement
stimdf = sdf.filter(pl.col('condition') == 'early_sd')
stimmeans = stimdf.group_by(['subject', 'store']).agg(pl.col('bandpower_rel').mean()).sort(['subject', 'store'])

In [ ]:
fig_id = 'delta_during_early-sd'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

nnxo = stimmeans.filter(pl.col('store') == 'NNXo')['bandpower_rel'].to_numpy()
nnxr = stimmeans.filter(pl.col('store') == 'NNXr')['bandpower_rel'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.show()
f, ax = acr.plots.gen_paired_boxplot(nnxr/nnxr, nnxo/nnxr, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
ax.set_xticklabels(['Contra. Control', 'Optrode'])
#ax.set_ylim(0.7, 4.1)
plt.show()

f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
conds = ['circ_bl']
sdf = relbp.filter(pl.col('condition').is_in(conds)).filter(pl.col('band')=='delta')
stimdf = sdf.filter(pl.col('condition') == 'circ_bl')
stimmeans = stimdf.group_by(['subject', 'store']).agg(pl.col('bandpower_rel').mean()).sort(['subject', 'store'])

In [ ]:
fig_id = 'delta_during_circ-bl'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

nnxo = stimmeans.filter(pl.col('store') == 'NNXo')['bandpower_rel'].to_numpy()
nnxr = stimmeans.filter(pl.col('store') == 'NNXr')['bandpower_rel'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.show()
f, ax = acr.plots.gen_paired_boxplot(nnxr/nnxr, nnxo/nnxr, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
ax.set_xticklabels(['Contra. Control', 'Optrode'])
#ax.set_ylim(0.7, 4.1)
plt.show()

f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

# REM

In [ ]:
rdfs = []
for subject in subjects:
    subdf = relbp.filter(pl.col('subject') == subject)
    reb_start = hyp_dicts[subject]['rebound']['start_time'].min()
    reb_end = hyp_dicts[subject]['rebound']['end_time'].max()
    subdf = subdf.ts(reb_start, reb_end).filter(pl.col('state') == 'REM')
    rdfs.append(subdf)
rdf = pl.concat(rdfs)

In [ ]:
rdf_theta = rdf.filter(pl.col('band') == 'theta').group_by(['subject', 'store']).agg(pl.col('bandpower_rel').mean()).sort(['subject', 'store'])
fig_id = 'REM__theta_during_rebound'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

nnxo = rdf_theta.filter(pl.col('store') == 'NNXo')['bandpower_rel'].to_numpy()
nnxr = rdf_theta.filter(pl.col('store') == 'NNXr')['bandpower_rel'].to_numpy()
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.show()
f, ax = acr.plots.gen_paired_boxplot(nnxr/nnxr, nnxo/nnxr, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
ax.set_xticklabels(['Contra. Control', 'Optrode'])
#ax.set_ylim(0.7, 4.1)
plt.show()

f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

#stats = pg.ttest(nnxr, nnxo, paired=True)
stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'rbc/r', [stats['RBC'][0], r])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

# FULL BAND SPECTRUM

In [ ]:
fig_id = 'NREM_THETA_during_rebound'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'
BAND = 'theta'
data = reb_avgs.query('band == @BAND')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values
theta_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.show()

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
fig_id = 'NREM_alpha_during_rebound'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'
BAND = 'alpha'
data = reb_avgs.query('band == @BAND')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values
alpha_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.show()

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
fig_id = 'NREM_sigma_during_rebound'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'
BAND = 'sigma'
data = reb_avgs.query('band == @BAND')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values
sigma_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.show()

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
fig_id = 'NREM_beta_during_rebound'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'
BAND = 'beta'
data = reb_avgs.query('band == @BAND')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values
beta_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.show()

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
fig_id = 'NREM__theta_alpha_sigma_beta-COMBO'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

f, ax = plt.subplots(figsize=(12, 4))
ax.set_xlim(0.15, 0.9)

acr.plots.add_paired_lines(ax, sigma_rel, beta_rel, 0.65, 0.85, colors=[SOM_BLUE, NNXR_GRAY], alpha=0.3)
acr.plots.add_paired_lines(ax, theta_rel, alpha_rel, 0.25, 0.45, colors=[SOM_BLUE, NNXR_GRAY], alpha=0.3)
acr.plots.add_paired_lines(ax, alpha_rel, sigma_rel, 0.45, 0.65, colors=[SOM_BLUE, NNXR_GRAY], alpha=0.3)

acr.plots.add_boxplot(ax, theta_rel, positions=[0.2], color=SOM_BLUE, alpha=1)
acr.plots.add_data_points(ax, theta_rel, x_pos=0.25, color=SOM_BLUE)

acr.plots.add_boxplot(ax, alpha_rel, positions=[0.4], color=SOM_BLUE, alpha=1)
acr.plots.add_data_points(ax, alpha_rel, x_pos=0.45, color=SOM_BLUE)



acr.plots.add_boxplot(ax, sigma_rel, positions=[0.6], color=SOM_BLUE, alpha=1)
acr.plots.add_data_points(ax, sigma_rel, x_pos=0.65, color=SOM_BLUE)



acr.plots.add_boxplot(ax, beta_rel, positions=[0.80], color=SOM_BLUE, alpha=1)
acr.plots.add_data_points(ax, beta_rel, x_pos=0.85, color=SOM_BLUE)



ax.set_xticks([0.2, 0.4, 0.6, 0.8])
ax.set_xticklabels(['Theta', 'Alpha', 'Sigma', 'Beta'])


ax.set_ylim(0.9, 1.1)
ax.axhline(1, color='red', linestyle='--', linewidth=3)

f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# Subgranular vs Infra-Granular

In [ ]:
plt.rcdefaults()
acr.plots.supl()
nbroot = nbroot = f'{acr.utils.PAPER_FIGURE_ROOT}/swa/by_depth'

In [ ]:
relbp = acr.info_pipeline.label_df_sub_infra(relbp, join_on=['subject', 'store', 'channel'])

In [ ]:
rebbp = relbp.filter(pl.col('condition')=='rebound')
reb_avgs = rebbp.group_by(['subject', 'store', 'band', 'label']).mean().to_pandas()

In [ ]:
BAND = 'delta'
label = 'infragranular'

fig_id = f'NREM_{BAND}_during_rebound--{label}'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

data = reb_avgs.query('band == @BAND').query('label == @label')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values
sigma_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.show()

f, ax = acr.plots.gen_paired_boxplot(nnxr/nnxr, nnxo/nnxr, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)
plt.show()

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats


In [ ]:
BAND = 'delta'
label = 'granular'

fig_id = f'NREM_{BAND}_during_rebound--{label}'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

data = reb_avgs.query('band == @BAND').query('label == @label')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values
sigma_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.show()

f, ax = acr.plots.gen_paired_boxplot(nnxr/nnxr, nnxo/nnxr, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
ax.set_xticklabels(['Contra. Control', 'Optrode'])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)
plt.show()
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
BAND = 'delta'
label = 'subgranular'

fig_id = f'NREM_{BAND}_during_rebound--{label}'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{nbroot}/{fig_name}.png'

data = reb_avgs.query('band == @BAND').query('label == @label')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values
sigma_rel = nnxo/nnxr
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.show()

f, ax = acr.plots.gen_paired_boxplot(nnxr/nnxr, nnxo/nnxr, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True)
ax.set_xticklabels(['Contra. Control', 'Optrode'])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)
plt.show()

# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
%connect_info